In [ ]:
import os
import time
import json
import logging
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from pythonProject.get_from_file import read_excel
from pythonProject.get_from_api import fetch_data
from actions import comparer_json, send_json_data

# Configuration du logger
log_file = "log.txt"
logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

# Définition des fichiers et de l'API
excel_file = "data.xlsx"
sheet_name = "Sheet1"
api_url = "https://essential-orelie-devpro13-5241d920.koyeb.app/kapi"
pending_file = "pending_data.json"

def process_data():
    """Vérifie et compare les données avant d'envoyer les mises à jour."""
    logging.info(f"🔄 Vérification des données à partir de {excel_file}...")

    try:
        # 1️⃣ Récupération des données depuis le fichier Excel
        excel_data = read_excel(excel_file, sheet_name)
        if excel_data is None:  # Gérer le cas où la fonction retourne None
            logging.error("❌ Erreur lors de la lecture du fichier Excel.")
            excel_data = []  # On met une liste vide

        with open("excel_content_data.json", "w", encoding="utf-8") as excel_file_data:
            json.dump(excel_data, excel_file_data, indent=4, ensure_ascii=False)

        logging.info(f"✅ Données Excel enregistrées dans excel_content_data.json ({len(excel_data)} éléments)")

        # 2️⃣ Récupération des données depuis l'API
        api_data = fetch_data(api_url)
        if api_data is None:  # Vérifier si l'API renvoie des données
            logging.error("❌ Aucune donnée récupérée depuis l'API.")
            api_data = []  # On met une liste vide

        with open("content_data.json", "w", encoding="utf-8") as api_file:
            json.dump(api_data, api_file, indent=4, ensure_ascii=False)

        logging.info(f"✅ Données API enregistrées dans content_data.json ({len(api_data)} éléments)")

        # 3️⃣ Comparaison des données et génération du fichier pending_data.json
        comparer_json("content_data.json", "excel_content_data.json", pending_file)

        # 4️⃣ Vérification et envoi des données si `pending_data.json` existe
        if os.path.exists(pending_file):
            with open(pending_file, "r", encoding="utf-8") as f:
                pending_data = json.load(f)

            if pending_data:  # Vérification si le fichier contient des données
                logging.info("🆕 Nouvelles données détectées, envoi en cours...")
                send_json_data(f"{api_url}/ask-kapi", pending_file)

                # ✅ Suppression du fichier après envoi
                os.remove(pending_file)
                logging.info("🗑️ Fichier pending_data.json supprimé après envoi.")
            else:
                logging.info("✅ Aucun nouvel élément à envoyer. Suppression du fichier.")
                os.remove(pending_file)  # Suppression si vide
        else:
            logging.info("❌ Le fichier pending_data.json n'a pas été généré.")

    except Exception as e:
        logging.error(f"🚨 Erreur lors du traitement des données : {e}")

class ExcelFileHandler(FileSystemEventHandler):
    """Classe pour gérer les événements de modification du fichier Excel."""

    def on_modified(self, event):
        if event.src_path == os.path.abspath(excel_file):
            logging.info(f"📂 Modification détectée sur {excel_file}, relance du traitement...")
            process_data()

# 🔹 Exécution initiale au démarrage
process_data()

# 🔹 Initialisation de l'observateur de fichiers
event_handler = ExcelFileHandler()
observer = Observer()
observer.schedule(event_handler, path=os.path.dirname(os.path.abspath(excel_file)), recursive=False)

logging.info(f"👀 Surveillance du fichier {excel_file} en cours...")
observer.start()

try:
    while True:
        time.sleep(1)  # Garde le programme actif
except KeyboardInterrupt:
    logging.info("🛑 Arrêt du programme...")
    observer.stop()
    observer.join()


✅ Aucun nouvel élément trouvé. Aucune action requise.
